In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d

# Define log-normal distribution function
def log_normal(x, mu, sigma, A):
    return A / (x * sigma * np.sqrt(2 * np.pi)) * np.exp(- (np.log(x) - mu)**2 / (2 * sigma**2))

# Input data file paths (tab-delimited)
file_paths = [
    'path/to/data/file1.txt',
    'path/to/data/file2.txt',
    'path/to/data/file3.txt'
]

# Load data from each file
data_arrays = [np.genfromtxt(f, delimiter='\t', dtype=np.float64, missing_values='', filling_values=np.nan) for f in file_paths]

# Find shortest data length to align arrays
min_length = min(array.shape[0] for array in data_arrays)
trimmed_data_arrays = [array[:min_length] for array in data_arrays]

# Compute mean across datasets
mean_data = np.mean(trimmed_data_arrays, axis=0)

# Invert signal if necessary (example: negative intensities to positive)
mean_data[:, 1] = -mean_data[:, 1]

# Extract and filter data
x_data = mean_data[:, 0] / 2  # Optional: time rescaling
y_data = mean_data[:, 1]

# Keep only valid x range (e.g., first 30 minutes)
valid_indices = (x_data > 0) & (x_data <= 1800)
x_data = x_data[valid_indices]
y_data = y_data[valid_indices]

# Prevent log(0) errors by adding small offset
epsilon = 1e-10
x_data += epsilon

# Handle NaNs and infinities
finite_indices = np.isfinite(y_data)
x_finite = x_data[finite_indices]
y_finite = y_data[finite_indices]

# Interpolate missing values
interp_func = interp1d(x_finite, y_finite, bounds_error=False, fill_value="extrapolate")
y_data[~finite_indices] = interp_func(x_data[~finite_indices])

# Initial parameter guess for curve fitting
initial_guess = [np.mean(np.log(x_data)), np.std(np.log(x_data)), np.max(y_data)]

# Fit data to log-normal function
popt, pcov = curve_fit(log_normal, x_data, y_data, p0=initial_guess, maxfev=100000)

# Generate fitted curve
y_fit = log_normal(x_data, *popt)

# Calculate R²
ss_res = np.sum((y_data - y_fit) ** 2)
ss_tot = np.sum((y_data - np.mean(y_data)) ** 2)
r_squared = 1 - (ss_res / ss_tot)

# Unpack fitted parameters
mu, sigma, A = popt

# Print fitted equation and parameters
print(f'Fitted log-normal parameters: mu = {mu:.3f}, sigma = {sigma:.3f}, A = {A:.3f}')
print(f'Fitted function: A / (x * σ * sqrt(2π)) * exp(-(log(x) - μ)² / (2σ²))')

# Plot result
plt.figure(figsize=(10, 6))
plt.plot(x_data - epsilon, y_data, 'r-', label='Averaged Signal')
plt.plot(x_data - epsilon, y_fit, 'b--', label='Log-normal Fit: μ=%.3f, σ=%.3f, A=%.3f' % tuple(popt))
plt.axhline(y=0, color='black', linewidth=0.8)
plt.xlabel('Time (seconds)')
plt.ylabel('Normalized Intensity (I - I₀) / I₀')
plt.title('Averaged Signal with Log-normal Fit')
plt.text(15, 0.05, f'R² = {r_squared:.3f}', fontsize=12)
plt.legend()
plt.tight_layout()
plt.show()

# Report R²
print(f'R²: {r_squared:.3f}')
